In [13]:
import datetime

class Timer:
    """A simple timer class"""

    def __init__(self):
        pass

    def start(self):
        """Starts the timer"""
        self.start = datetime.datetime.now()
        return self.start

    def stop(self, message="Total: "):
        """Stops the timer.  Returns the time elapsed"""
        self.stop = datetime.datetime.now()
        return message + str(self.stop - self.start)

    def now(self, message="Now: "):
        """Returns the current time with a message"""
        return message + ": " + str(datetime.datetime.now())

    def elapsed(self, message="Elapsed: "):
        """Time elapsed since start was called"""
        return message + str(datetime.datetime.now() - self.start)

    def split(self, message="Split started at: "):
        """Start a split timer"""
        self.split_start = datetime.datetime.now()
        return message + str(self.split_start)

    def unsplit(self, message="Unsplit: "):
        """Stops a split. Returns the time elapsed since split was called"""
        return message + str(datetime.datetime.now() - self.split_start)


In [4]:
import pandas as pd
import sqlite3
import os
from numpy import random

from pandas.io import sql

df = pd.DataFrame(random.randn(1000000,2),columns=list('AB'))

def test_sql_write(df):
    if os.path.exists('test.sql'):
        os.remove('test.sql')
    sql_db = sqlite3.connect('test.sql')
    df.to_sql(name='test_table', con=sql_db)
    sql_db.close()

def test_sql_read():
    sql_db = sqlite3.connect('test.sql')
    pd.read_sql_query("select * from test_table", sql_db)
    sql_db.close()

def test_hdf_fixed_write(df):
    df.to_hdf('test_fixed.hdf','test',mode='w')

def test_hdf_fixed_read():
    pd.read_hdf('test_fixed.hdf','test')

def test_hdf_fixed_write_compress(df):
    df.to_hdf('test_fixed_compress.hdf','test',mode='w',complib='blosc')

def test_hdf_fixed_read_compress():
    pd.read_hdf('test_fixed_compress.hdf','test')

def test_hdf_table_write(df):
    df.to_hdf('test_table.hdf','test',mode='w',format='table')

def test_hdf_table_read():
    pd.read_hdf('test_table.hdf','test')

def test_hdf_table_write_compress(df):
    df.to_hdf('test_table_compress.hdf','test',mode='w',complib='blosc',format='table')

def test_hdf_table_read_compress():
    pd.read_hdf('test_table_compress.hdf','test')

def test_csv_write(df):
    df.to_csv('test.csv',mode='w')

def test_csv_read():
    pd.read_csv('test.csv',index_col=0)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 2 columns):
A    1000000 non-null float64
B    1000000 non-null float64
dtypes: float64(2)
memory usage: 15.3 MB


In [6]:
%timeit test_sql_write(df)
%timeit test_hdf_fixed_write(df)
%timeit test_hdf_fixed_write_compress(df)
%timeit test_hdf_table_write(df)
%timeit test_hdf_table_write_compress(df)
%timeit test_csv_write(df)

1 loop, best of 3: 4.53 s per loop
10 loops, best of 3: 28 ms per loop
10 loops, best of 3: 100 ms per loop
1 loop, best of 3: 405 ms per loop
1 loop, best of 3: 531 ms per loop
1 loop, best of 3: 4.53 s per loop


In [30]:
%timeit test_sql_read()
%timeit test_hdf_fixed_read()
%timeit test_hdf_fixed_read_compress()
%timeit test_hdf_table_read()
%timeit test_hdf_table_read_compress()
%timeit test_csv_read()

1 loop, best of 3: 491 ms per loop
10 loops, best of 3: 40.5 ms per loop
10 loops, best of 3: 44 ms per loop
1 loop, best of 3: 631 ms per loop
1 loop, best of 3: 733 ms per loop
1 loop, best of 3: 185 ms per loop


In [18]:
### FIXED
timer = Timer()
timer.start()
file = "/Users/manuel/development/thesis/overlap/filtered_hg19DNase_H3K27ac_FANTOM_overlapped.csv"
df = pd.DataFrame.from_csv(file, sep="\t")
df.reset_index(level=0, inplace=True)
print("Loaded csv file to dataframe in", timer.elapsed())
timer.split()
df.to_hdf('encode_fantom.hdf','encode_fantom',mode='w')
print("Saved dataframe to hdf fixed table in", timer.unsplit())
timer.split()
returned = pd.read_hdf('encode_fantom.hdf','encode_fantom')
print("Loaded hdf table in", timer.unsplit())

Loaded csv file to dataframe in Elapsed: 0:00:09.146367
Saved dataframe to hdf table in Unsplit: 0:00:03.834763
Loaded hdf table in Unsplit: 0:00:02.324402


In [22]:
### TABLE (query are allowed)
timer = Timer()
timer.start()
file = "/Users/manuel/development/thesis/overlap/filtered_hg19DNase_H3K27ac_FANTOM_overlapped.csv"
df = pd.DataFrame.from_csv(file, sep="\t")
df.reset_index(level=0, inplace=True)
print("Loaded csv file to dataframe in", timer.elapsed())
timer.split()
df.to_hdf('encode_fantom_table.hdf','encode_fantom', mode='w', format='table', data_columns=['biosample_term_name'])
print("Saved dataframe to hdf table in", timer.unsplit())
timer.split()
returned = pd.read_hdf('encode_fantom_table.hdf','encode_fantom')
print("Loaded hdf table in", timer.unsplit())
returned.info()

Loaded csv file to dataframe in Elapsed: 0:00:09.678734
Saved dataframe to hdf table in Unsplit: 0:00:38.186737
Loaded hdf table in Unsplit: 0:00:39.275191
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1801781 entries, 0 to 1801780
Data columns (total 27 columns):
chrom                  object
start                  int64
end                    int64
name                   object
score                  int64
strand                 object
size                   int64
method                 object
description            object
assembly               object
biosample_type         object
biosample_term_id      object
biosample_term_name    object
developmental_slims    object
system_slims           object
organ_slims            object
encyclopedia           object
FA_chrom               object
FA_start               int64
FA_end                 int64
FA_name                object
FA_score               int64
FA_size                int64
FA_method              object
FA_ovlp_len        

In [26]:
timer = Timer()
timer.start()
placenta_df = pd.read_hdf('encode_fantom_table.hdf','encode_fantom',where='biosample_term_name == "placenta"')
print("Query on hdf table by biosample_term_name == \"placenta\" in", timer.elapsed())
timer.stop()

Query on hdf table by biosample_term_name == "placenta" in Elapsed: 0:00:00.787150


'Total: 0:00:00.787254'

In [25]:
placenta_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38752 entries, 36794 to 75545
Data columns (total 27 columns):
chrom                  38752 non-null object
start                  38752 non-null int64
end                    38752 non-null int64
name                   38752 non-null object
score                  38752 non-null int64
strand                 38752 non-null object
size                   38752 non-null int64
method                 38752 non-null object
description            38752 non-null object
assembly               38752 non-null object
biosample_type         38752 non-null object
biosample_term_id      38752 non-null object
biosample_term_name    38752 non-null object
developmental_slims    38752 non-null object
system_slims           38752 non-null object
organ_slims            38752 non-null object
encyclopedia           38752 non-null object
FA_chrom               38752 non-null object
FA_start               38752 non-null int64
FA_end                 38752 non-null